In [1]:
import os
from typing import Optional
import wave
import numpy as np
import sys

In [2]:
sys.path.append('/Users/jaewone/developer/tensorflow/baby-cry-classification')

In [3]:
from constant.os import *
from utils.os import *
from utils.sound import *

In [4]:
# wav 파일을 읽어온다.
with wave.open("/Users/jaewone/Downloads/hungry_9.wav", "r") as file:
    params = file.getparams()
    n_channels, sampwidth, framerate, n_frames = params[:4]
    audio_data = file.readframes(n_frames)
    wave_data = np.frombuffer(audio_data, dtype=np.int16)

# 오디오 시그널을 통한 에너지 계산
energy = np.abs(wave_data)

# 백색 잡음을 분류하기 위한 임계값을 설정.
# 전체 에너지의 하위 10%에 10을 곱하여 임계값을 설정하였으나 추가적은 고민이 필요하다.
threshold = np.percentile(energy, 10) * 10

In [5]:
def detect_non_silence(audio_data, threshold, frame_size):
    """
    오디오 신호에서 무음이 아닌 섹션의 시작과 끝을 감지한다.
    
    Parameters:
        - audio_data (numpy.array): 묵음을 감지해야 하는 오디오 데이터.
        - threshold (float): 오디오가 무음으로 간주되는 에너지 임계값.
        - frame_size (int): 오디오 에너지의 이동 평균을 계산하기 위해 고려할 샘플 수.
    
    Returns:
        - start (int): 비침묵 섹션의 시작 샘플.
        - end (int): non-silence 섹션의 엔딩 샘플.
    """
    
    moving_avg = np.convolve(audio_data, np.ones((frame_size,))/frame_size, mode='valid')
    non_silence = np.where(moving_avg > threshold)[0]

    start = non_silence[0]
    end = non_silence[-1] + frame_size  # compensate for the 'valid' mode in convolution

    return start, end

# Use a larger frame size to get a moving average of the audio energy
frame_size = 5000
start, end = detect_non_silence(energy, threshold, frame_size)
trimmed_wave_data_2 = wave_data[start:end]

start, end

(36003, 274324)

In [6]:
# trim된 numpy array를 wav 파일로 저장한다.
output_path_2 = "/Users/jaewone/Downloads/hungry_9_trimmed_me.wav"
with wave.open(output_path_2, "w") as out_file:
    out_file.setparams(params)
    out_file.writeframes(trimmed_wave_data_2.tobytes())

output_path_2

'/Users/jaewone/Downloads/hungry_9_trimmed_me.wav'